In [42]:
import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
import tensorflow_hub as hub
from PIL import Image, ImageDraw, ImageFont
from os import path, listdir
from pathlib import Path
import cv2 as cv

%matplotlib inline

In [68]:
class Model():
      
  def load_model(self):
    self.__model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
    
  def __init__(self):
    self.load_model()

  def pred(self, input_image):
    return self.__model(input_image.get_data())

In [74]:
class Input_image():

  def __init__(self,path):
     self.set_data(np.array([plt.imread(path)]))
  
  def set_data(self,image):
    self.__data = image

  def get_data(self):
    return self.__data

  def get_draw(self):
    return ImageDraw.Draw(Image.open(self.__data))

In [11]:
class Utils():

  @staticmethod
  def add_blocks(pred,draw):
    boxes = pred['detection_boxes'].numpy()[0]
    classes = pred['detection_classes'].numpy()[0]

    height, width = image_data.shape[1],image_data.shape[2]
    boxes[:, 0] *= height
    boxes[:, 1] *= width
    boxes[:, 2] *= height
    boxes[:, 3] *= width
    
    for i in range(len(boxes)):
      if pred['detection_scores'].numpy()[0][i] > PRECISION:
        draw.rectangle(prepare_coords(boxes[i]), outline = which_color(classes[i]), width = 3)
        draw.text((boxes[i][1], boxes[i][0]),str(classes[i])[:-2], fill=(255,255,255), stroke_fill= (0,0,0,255), stroke_width = 2, font= ImageFont.truetype("Roboto-Regular.ttf", 20))

    @staticmethod
    def which_color(class_id):
      color_value = int(class_id) * 9
      return (min(color_value, 255), max(min(color_value - 255, 255),0),max(min(color_value - 256 * 2 - 1, 255),0))

    @staticmethod
    def prepare_coords(array):
      return (array[1], array[0], array[3], array[2])

In [75]:
class Glie_44():

  def __init__(self, model, precision = .2):
    self.__model = model
    self.__precision = precision

  def run_on_image(self, image_path):
    image = Input_image(image_path)
    pred = self.__model.pred(image)

    draw = image.get_draw()
    Utils.add_blocks(draw)

    return np.array(image)
  
  def run_on_folder(self, folder_path = Path().absolute().joinpath('Data/VisDrone2019-MOT-train/sequences/uav0000013_00000_v'), output_folder = Path().absolute().joinpath('outputs')):
    video_frames = sorted(listdir(folder_path))
    s = Image.open(folder_path.joinpath(video_frames[0])).size

    output_path = path.basename(folder_path) + '.avi'
    fourcc = cv.VideoWriter_fourcc(*'DIVX')
    writer = cv.VideoWriter(str(output_folder.joinpath(output_path).stem), fourcc, 20, s)

    sample = video_frames
    total_frames = len(sample)
    for i in range(total_frames):
      #print(i + 1, '/', total_frames)
      result = self.run_on_image(folder_path.joinpath(sample[i]))
      print(sample[i], ':', result.shape)
      writer.write(result)

    writer.release()

In [35]:
model = Model()

In [76]:
glie = Glie_44(model = model)
glie.run_on_folder()

AttributeError: ignored